### Step 1: Mount Google Drive
To access the dataset stored in Google Drive, we first mount the drive using `drive.mount()`. This ensures that our script can read data directly from the cloud without needing to upload files manually each time.


### Step 2: Load Datasets
We load two CSV files:  
- **MusicInfo.csv**: Contains metadata about songs, such as genre, duration, and various musical attributes.
- **charts.csv**: Contains streaming data, including the number of streams, song title, artist, and ranking information.

The datasets are read into Pandas DataFrames for further processing.


### Step 3: Data Cleaning
Since data inconsistencies can affect our analysis, we perform several cleaning steps:
1. **Standardizing Column Names**  
   - We rename `"name"` to `"title"` in `MusicInfo.csv` for consistency.
  
2. **Lowercasing Text Columns**  
   - We convert `title` and `artist` columns to lowercase in both datasets to avoid case-sensitive mismatches.

3. **Cleaning Song Titles**  
   - We remove **parentheses** and special characters from song titles using a regex function.

4. **Dropping Unnecessary Columns**  
   - Several features from `MusicInfo.csv` (e.g., `danceability`, `energy`, `tempo`, etc.) are dropped as they are not needed for our analysis.

5. **Handling Duplicates**  
   - We remove duplicate songs based on `title` and `artist`, keeping only the first occurrence.

6. **Converting `streams` Column to Numeric**  
   - Ensures that streaming counts are treated as numbers, allowing calculations like averages or sums.

7. **Handling Missing Values**  
   - We drop rows where `title`, `artist`, or `streams` contain `NaN` values.

8. **Extracting Primary Genre**  
   - Since a song may belong to multiple genres, we extract only the first genre listed.


In [ ]:
import pandas as pd
import re
import time
import numpy as np
import os
from collections import Counter
from google.colab import drive

# === Step 1: Mount Google Drive ===
drive.mount('/content/drive', force_remount=True)

# === Step 2: Load Datasets ===
music_info_path = "/content/drive/My Drive/MusicInfo.csv"
charts_path = "/content/drive/My Drive/charts.csv"

# Load datasets
music_info_df = pd.read_csv(music_info_path)
charts_df = pd.read_csv(charts_path)

# === Step 3: Data Cleaning ===
# Standardize column names
music_info_df.rename(columns={"name": "title"}, inplace=True)

# Convert titles and artists to lowercase
music_info_df["title"] = music_info_df["title"].str.lower().str.strip()
music_info_df["artist"] = music_info_df["artist"].str.lower().str.strip()
charts_df["title"] = charts_df["title"].str.lower().str.strip()
charts_df["artist"] = charts_df["artist"].str.lower().str.strip()

# Function to clean song titles
def clean_song_title(title):
    title = re.sub(r"\(.*?\)", "", title)  # Remove text inside parentheses
    title = re.sub(r"[^a-z0-9 ]", "", title)  # Remove special characters
    return title.strip()

# Apply cleaning function to charts dataset
charts_df["title"] = charts_df["title"].apply(clean_song_title)

# Remove unnecessary columns
charts_df.drop(columns=["date"], inplace=True, errors="ignore")
music_info_df.drop(columns=["duration_ms", "danceability", "energy", "key", "loudness",
                            "mode", "speechiness", "acousticness", "instrumental", "liveness",
                            "valence", "tempo", "time_signature", "tags", "instrumentalness",
                            "spotify_id"], inplace=True, errors="ignore")

# Remove duplicates
charts_df.drop_duplicates(subset=["title", "artist"], keep="first", inplace=True)
music_info_df.drop_duplicates(subset=["title", "artist"], keep="first", inplace=True)

# Convert 'streams' column to numeric
charts_df["streams"] = pd.to_numeric(charts_df["streams"], errors="coerce")

# Remove missing values
charts_df.dropna(subset=["title", "artist", "streams"], inplace=True)
music_info_df.dropna(subset=["title", "artist"], inplace=True)

# Extract first genre
music_info_df["genre"] = music_info_df["genre"].apply(lambda x: x.split(",")[0] if isinstance(x, str) else x)


Mounted at /content/drive


### Step 4: Merging Datasets
Since we have two datasets—**charts.csv** and **MusicInfo.csv**—we need to merge them to perform further analysis.

- We use an **inner join** on `title` and `artist` to ensure only songs appearing in both datasets are included.
- The `how="inner"` parameter ensures that unmatched rows are **excluded** from the final dataset.
- We track the **execution time**

### Step 5: Data Transformation
After merging, we select only the **relevant columns** for analysis:
- `title`: Name of the song
- `artist`: Artist name
- `streams`: Number of times the song was streamed
- `region`: The region where the song was popular
- `genre`: Genre classification of the song
- `year`: Year of the song's release
- `chart`: The type of ranking (`Top 200` or `Viral 50`)

This step ensures we work with only the necessary information, making further analysis **faster and more efficient**.


In [ ]:
# === Step 4: Merge Datasets ===
start_time_merge = time.time()
df_merged = pd.merge(charts_df, music_info_df, on=["title", "artist"], how="inner")
merge_time = time.time() - start_time_merge
print(f"Merge Execution Time: {merge_time:.2f} seconds")

# === Step 5: Data Transformation ===
df_analysis = df_merged[["title", "artist", "streams", "region", "genre", "year", "chart"]]

Merge Execution Time: 0.17 seconds


### 6.1: Find the Top 5 Most Streamed Songs Per Region

####  **Objective:**
To identify the **top 5 most streamed songs** for each region in the dataset.

In [ ]:
#  6.1: Find the top 5 most streamed songs per region

import time

start_time_top_songs = time.time()

# Find the top 5 most streamed songs per region
top_songs_per_region = (
    df_analysis.groupby("region")
    .apply(lambda x: x.nlargest(5, "streams"))
    .reset_index(drop=True)
)

top_songs_time = time.time() - start_time_top_songs

# Print results
print("Top 5 Most Streamed Songs Per Region:\n")
for region, group in top_songs_per_region.groupby("region"):
    print(f"🌍 Region: {region}")
    for _, row in group.iterrows():
        print(f"🎵 {row['title']} - {row['streams']} streams")
    print("-" * 50)  # Separator for readability

print(f"\nTop 5 Songs Per Region Execution Time: {top_songs_time:.5f} seconds")


Top 5 Most Streamed Songs Per Region:

🌍 Region: Argentina
🎵 dance monkey - 105980.0 streams
🎵 break my heart - 56313.0 streams
🎵 state of grace - 56192.0 streams
🎵 havana - 53982.0 streams
🎵 say so - 48348.0 streams
--------------------------------------------------
🌍 Region: Australia
🎵 violent crimes - 145695.0 streams
🎵 my bad - 136635.0 streams
🎵 stir fry - 132404.0 streams
🎵 8 - 101997.0 streams
🎵 rare - 99877.0 streams
--------------------------------------------------
🌍 Region: Austria
🎵 rooftop - 12982.0 streams
🎵 feliz navidad - 11698.0 streams
🎵 bad ideas - 10837.0 streams
🎵 griechischer wein - 8836.0 streams
🎵 river - 7459.0 streams
--------------------------------------------------
🌍 Region: Belgium
🎵 around the world - 12101.0 streams
🎵 lean on me - 11809.0 streams
🎵 gimme more - 8583.0 streams
🎵 think - 8096.0 streams
🎵 the ecstasy of gold - 8069.0 streams
--------------------------------------------------
🌍 Region: Brazil
🎵 head above water - 65171.0 streams
🎵 tell me y

<ipython-input-10-df98f61e2858>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(5, "streams"))


### 6.2: Find the Most Common Genre Per Region

####  **Objective:**
To determine the **most common music genre** in each region based on the dataset.

In [ ]:
import time
from collections import Counter

start_time_genre = time.time()

def most_common_genre(genres):
    all_genres = []
    for genre in genres.dropna():
        all_genres.append(genre)
    return Counter(all_genres).most_common(1)[0][0] if all_genres else None

common_genre_per_region = df_analysis.groupby("region")["genre"].agg(most_common_genre)

genre_time = time.time() - start_time_genre

# Print all regions with their most common genres
print("Most Common Genre Per Region:")
for region, genre in common_genre_per_region.items():
    print(f"{region}: {genre}")

print(f"\nMost Common Genre Per Region Execution Time: {genre_time:.5f} seconds")


Most Common Genre Per Region:
Argentina: Rock
Australia: Rock
Austria: Rock
Belgium: Rock
Brazil: None
Bulgaria: Rap
Canada: Rock
Chile: Rock
Colombia: None
Costa Rica: Rock
Czech Republic: Rock
Denmark: Pop
Dominican Republic: None
Ecuador: RnB
Estonia: Electronic
Finland: Metal
France: Rock
Greece: Rock
Honduras: None
Hong Kong: RnB
Hungary: RnB
Iceland: Rock
India: Rock
Indonesia: Rock
Ireland: Rock
Israel: World
Italy: None
Japan: Rock
Latvia: Rock
Lithuania: None
Malaysia: RnB
Mexico: Latin
Netherlands: Rock
New Zealand: Rock
Norway: Rock
Paraguay: Rock
Peru: RnB
Philippines: Pop
Poland: Rock
Portugal: None
Romania: None
Russia: Pop
Saudi Arabia: Rap
Singapore: Rock
Slovakia: None
South Africa: Pop
Spain: None
Sweden: Rock
Taiwan: Jazz
Thailand: None
Turkey: None
Ukraine: Rap
United Arab Emirates: None
United Kingdom: Rock
United States: Pop
Uruguay: Blues
Vietnam: Rock

Most Common Genre Per Region Execution Time: 0.06510 seconds


### 6.3: Compute the Average Streams Per Region

#### **Objective:**
To calculate the **average number of streams** per song for each region.

In [ ]:
#  6.3: Compute the average streams per region
import time

start_time_avg_streams = time.time()

# Compute the average streams per region
avg_streams_per_region = df_analysis.groupby("region")["streams"].mean().sort_values(ascending=False)

avg_streams_time = time.time() - start_time_avg_streams

# Print all regions with their average streams
print("Average Streams Per Region:")
for region, avg_streams in avg_streams_per_region.items():
    print(f"{region}: {avg_streams:.2f}")

print(f"\nAverage Streams Per Region Execution Time: {avg_streams_time:.5f} seconds")

Average Streams Per Region:
United States: 254107.92
Mexico: 56482.92
Brazil: 48299.12
United Kingdom: 47895.30
Canada: 34703.28
Australia: 34223.62
Sweden: 33824.97
France: 32951.20
Italy: 31185.25
Argentina: 30495.32
Netherlands: 22653.86
Turkey: 21251.00
Philippines: 20291.55
Spain: 19356.50
Indonesia: 15511.00
Chile: 15356.13
Norway: 14493.42
Denmark: 13653.17
Russia: 11707.00
Poland: 9763.42
Finland: 8417.76
Colombia: 7552.50
Peru: 7324.43
Singapore: 5699.92
New Zealand: 5564.26
Belgium: 4818.68
Ireland: 4702.42
Malaysia: 4594.00
India: 4327.00
Austria: 4143.23
Taiwan: 3581.80
Portugal: 3485.50
Hong Kong: 3484.83
Ukraine: 3475.00
Costa Rica: 2927.34
Japan: 2644.79
Ecuador: 2463.50
Dominican Republic: 2347.00
South Africa: 2139.00
Saudi Arabia: 2118.50
Vietnam: 2105.00
Romania: 1999.33
Uruguay: 1985.33
Czech Republic: 1938.48
Hungary: 1712.39
Paraguay: 1632.36
Greece: 1532.00
Iceland: 1459.42
United Arab Emirates: 1402.00
Bulgaria: 1395.25
Thailand: 1315.00
Latvia: 1258.73
Estonia:

### 6.4: Compute the Count of Songs in Each Region for "Top 200" and "Viral 50" Charts

####  **Objective:**
To count the number of songs that appear in either the **"Top 200" or "Viral 50"** charts for each region.


In [ ]:
#  6.4: Compute the count of songs in each region for "Top 200" and "Viral 50" charts
import time

start_time_chart_counts = time.time()

# Compute the count of songs in each region for "Top 200" and "Viral 50" charts
chart_counts_per_region = (
    df_analysis[df_analysis["chart"].isin(["top200", "viral50"])]
    .groupby(["region", "chart"])
    .size()
    .unstack()
    .fillna(0)
)

chart_counts_time = time.time() - start_time_chart_counts

# Print all regions with their song counts for each chart
print("Chart Counts Per Region:")
print(chart_counts_per_region.to_string())  # Converts DataFrame to a readable string format

print(f"\nChart Counts Per Region Execution Time: {chart_counts_time:.5f} seconds")



Chart Counts Per Region:
chart                 top200
region                      
Argentina                 44
Australia                124
Austria                   31
Belgium                   50
Brazil                     8
Bulgaria                   4
Canada                    53
Chile                     15
Colombia                   6
Costa Rica                38
Czech Republic            25
Denmark                   12
Dominican Republic         1
Ecuador                    2
Estonia                    5
Finland                   33
France                    10
Greece                     4
Honduras                   1
Hong Kong                  6
Hungary                   18
Iceland                   24
India                      1
Indonesia                  4
Ireland                  164
Israel                     6
Italy                      4
Japan                     24
Latvia                    11
Lithuania                  1
Malaysia                  10
Mexico            

### 6.5: Find Which Region Has the Most Songs Appearing in *Both* Top 200 & Viral 50

####  **Objective:**
To identify the **region** with the highest number of songs appearing in **both the "Top 200" and "Viral 50"** charts.


In [ ]:
#  6.5: Find which region has the most songs appearing in *both Top 200 & Viral 50*
import time

start_time_common_songs = time.time()

# Filter for songs appearing in both "Top 200" and "Viral 50" charts
df_common_songs = df_analysis[df_analysis["chart"].isin(["top200", "viral50"])]

# Count occurrences per region and sort in descending order
common_chart_region_counts = df_common_songs.groupby("region")["title"].count().sort_values(ascending=False)

common_songs_time = time.time() - start_time_common_songs

# Print all regions with their song counts
print("Number of Songs Appearing in Both Top 200 & Viral 50 Per Region:")
for region, count in common_chart_region_counts.items():
    print(f"{region}: {count}")

# Identify the region with the most songs appearing in both charts
most_common_region = common_chart_region_counts.idxmax()
most_common_count = common_chart_region_counts.max()

print(f"\nRegion with the Most Common Songs: {most_common_region} ({most_common_count} songs)")

print(f"\nCommon Songs in Top 200 & Viral 50 Execution Time: {common_songs_time:.5f} seconds")


Number of Songs Appearing in Both Top 200 & Viral 50 Per Region:
Ireland: 164
United Kingdom: 131
Australia: 124
Canada: 53
Belgium: 50
Argentina: 44
Costa Rica: 38
Finland: 33
Austria: 31
Sweden: 30
Philippines: 29
United States: 25
Czech Republic: 25
Japan: 24
Poland: 24
Iceland: 24
Netherlands: 21
New Zealand: 19
Hungary: 18
Chile: 15
Taiwan: 15
Mexico: 13
Denmark: 12
Norway: 12
Singapore: 12
Paraguay: 11
Latvia: 11
Malaysia: 10
France: 10
Saudi Arabia: 8
Brazil: 8
Peru: 7
Portugal: 6
Colombia: 6
Hong Kong: 6
Israel: 6
South Africa: 5
Estonia: 5
Bulgaria: 4
Italy: 4
Greece: 4
Indonesia: 4
Romania: 3
Uruguay: 3
Thailand: 2
Turkey: 2
Vietnam: 2
Spain: 2
Ecuador: 2
Dominican Republic: 1
Russia: 1
Honduras: 1
Ukraine: 1
United Arab Emirates: 1
Slovakia: 1
Lithuania: 1
India: 1

Region with the Most Common Songs: Ireland (164 songs)

Common Songs in Top 200 & Viral 50 Execution Time: 0.00692 seconds


### 6.6: Identify the Year with the Most Songs in Top 200

####  **Objective:**
To determine which **year** had the most songs appearing in the **"Top 200"** chart.

In [ ]:
import time

start_time_top_year = time.time()

# Filter for Top 200 songs and count occurrences per year
top_year_songs = (
    df_analysis[df_analysis["chart"] == "top200"]
    .groupby("year")["title"]
    .count()
    .sort_values(ascending=False)
)

top_year_time = time.time() - start_time_top_year

# Print all years with their song counts
print("Most Songs in Top 200 by Year:")
for year, count in top_year_songs.items():
    print(f"{year}: {count}")

# Identify the year with the most songs in Top 200
most_common_year = top_year_songs.idxmax()
most_common_count = top_year_songs.max()

print(f"\n📅 Year with Most Songs in Top 200: {most_common_year} ({most_common_count} songs)")

print(f"\nMost Songs in Top 200 by Year Execution Time: {top_year_time:.5f} seconds")


Most Songs in Top 200 by Year:
2014: 92
2012: 79
2016: 79
2018: 76
2017: 73
2013: 72
2010: 66
2009: 65
2008: 57
2011: 56
2007: 46
2019: 40
2006: 38
2015: 37
2000: 26
2001: 23
2002: 21
2005: 20
2004: 20
2003: 20
1998: 13
2020: 10
1991: 10
1999: 8
1997: 8
1987: 7
1996: 7
1995: 5
1993: 5
1973: 5
1994: 5
1965: 3
1971: 3
1975: 3
1984: 3
1983: 2
1979: 2
1980: 2
1982: 2
1985: 2
1988: 2
1990: 2
1992: 2
1981: 1
1986: 1
1978: 1
1976: 1
1989: 1
1972: 1
1970: 1
1963: 1

📅 Year with Most Songs in Top 200: 2014 (92 songs)

Most Songs in Top 200 by Year Execution Time: 0.00306 seconds


### Step 7: Display Results

#### **Objective:**
To present the key findings from the dataset analysis, including:
1. **Top 5 Most Streamed Songs Per Region**
2. **Most Common Genre Per Region**
3. **Average Streams Per Region**
4. **Count of Top 200 & Viral 50 Songs Per Region**
5. **Regions with the Most Songs Appearing in Both Charts**

In [ ]:
# === Step 7: Display Results ===
print("📊 *Top 5 Songs Per Region:*", top_songs_per_region.head(10))
print("\n🎵 *Most Common Genre Per Region:*", common_genre_per_region)
print("\n💿 *Average Streams Per Region:*", avg_streams_per_region)
print("\n📈 *Count of Top 200 & Viral 50 Songs Per Region:*", chart_counts_per_region)
print("\n🌎 *Regions With Most Songs Appearing in Top 200 & Viral 50:*", common_chart_region_counts)

# === Step 8: Save Cleaned & Processed Files ===
charts_df.to_csv("/content/drive/My Drive/charts_cleaned.csv", index=False)
music_info_df.to_csv("/content/drive/My Drive/MusicInfo_cleaned.csv", index=False)
df_merged.to_csv("/content/drive/My Drive/merged_data.csv", index=False)




📊 *Top 5 Songs Per Region:*             title          artist   streams     region  genre  year   chart
0    dance monkey     tones and i  105980.0  Argentina    Rap  2019  top200
1  break my heart        dua lipa   56313.0  Argentina    NaN  2020  top200
2  state of grace    taylor swift   56192.0  Argentina    NaN  2012  top200
3          havana  camila cabello   53982.0  Argentina    NaN  2017  top200
4          say so        doja cat   48348.0  Argentina    NaN  2019  top200
5  violent crimes      kanye west  145695.0  Australia  Metal  2018  top200
6          my bad          khalid  136635.0  Australia    NaN  2019  top200
7        stir fry           migos  132404.0  Australia    NaN  2018  top200
8               8   billie eilish  101997.0  Australia    NaN  2019  top200
9            rare    selena gomez   99877.0  Australia    NaN  2020  top200

🎵 *Most Common Genre Per Region:* region
Argentina                     Rock
Australia                     Rock
Austria                 